In [1]:
#import data

import xlsxwriter
import pandas as pd
from pandas import DataFrame

path = ('/Users/mathijsvanboesschoten/Documents/Efteling/weer_rijen.xlsx')
data = pd.ExcelFile(path)


In [2]:
df1 = data.parse('weer (28:mei)')

# Drop rows with empty cells
df1.dropna(axis=0, how='any', thresh=None, subset=None, inplace=False)
df1.head()

,# STN,YYYYMMDD,FG,TG,TX,DR,RH,RHX,RHXH,NG,UG
0,350,19950101,51,18,36,35,35,7,12,5,89
1,350,19950102,51,26,54,39,46,14,6,5,85
2,350,19950103,15,-4,24,0,-1,-1,11,4,89
3,350,19950104,26,-38,3,0,0,0,1,0,93
4,350,19950105,46,-43,-15,0,-1,-1,24,4,74


In [3]:
# add month column 
df1['YYYYMMDD'] = pd.to_datetime(df1['YYYYMMDD'], format='%Y%m%d')
df1['month'] = df1['YYYYMMDD'].dt.month

In [4]:
#rename columns
df1 = df1.rename(index=str, columns={"   FG":"avg_windspeed","   TG":"avg_temp","   TX":"max_temp","   DR":"duur_neerslag","   RH":"som_neerslag_mm","  RHX":"hoogste_neerslag_in_uur"," RHXH":"uur_hoogste_neerslag","   NG":"avg_bewolking","   UG":"avg_percent_luchtvochtigheid"})

#change values
df1[['   FG','   TG','   TX','   DR','   RH','  RHX']] = df1[['   FG','   TG','   TX','   DR','   RH','  RHX']]/10

KeyError: "None of [Index(['   FG', '   TG', '   TX', '   DR', '   RH', '  RHX'], dtype='object')] are in the [columns]"

In [ ]:
df1.head()

Get average weather information per month over the past 20 years

In [ ]:
#group by month so we get average weather information over the past 20 years for each month
mean_weather_per_day = df1.groupby("month").mean()
mean_weather_per_day.columns = ['avg_#STN','avg_FG','avg_TG','avg_TX','avg_DR','avg_RH','avg_RHX','avg_RHXH','avg_UG']

In [ ]:
mean_weather_per_day.reset_index()

In [ ]:
#add average weather information to existing dataframe
df1_mean_weather = df1.merge(mean_weather_per_day, left_on='month', right_on='month')

In [ ]:
#create variables; change in temperature, change in windspeed and change in precipitation
df1_mean_weather['change_maxtemp'] = df1_mean_weather['max_temp'] - df1_mean_weather['avg_TX']
df1_mean_weather['change_avg_temp'] = df1_mean_weather['avg_temp'] - df1_mean_weather['avg_TG']
df1_mean_weather['change_avg_windspeed'] = df1_mean_weather['avg_windspeed'] - df1_mean_weather['avg_TX']
df1_mean_weather['change_duur_neerslag'] = df1_mean_weather['duur_neerslag'] - df1_mean_weather['avg_DR']
df1_mean_weather['change_som_neerslag_mm'] = df1_mean_weather['som_neerslag_mm'] - df1_mean_weather['avg_RH']

# drop average columns per month
df1_mean_weather = df1_mean_weather.drop(columns = ['month','# STN','avg_#STN', 'avg_FG','avg_TG', 'avg_TX', 'avg_DR', 'avg_RH', 'avg_RHX', 'avg_RHXH','avg_UG'])
df1_mean_weather.head()

## Write to excel

In [ ]:
# # Create a Pandas Excel writer using XlsxWriter as the engine.
# writer = pd.ExcelWriter('data_efteling_change_weather.xlsx', engine='xlsxwriter')

# # Convert the dataframe to an XlsxWriter Excel object.
# df1_mean_weather.to_excel(writer, sheet_name='data')

# # Close the Pandas Excel writer and output the Excel file.
# writer.save()